In [6]:
pip install thermo



  Obtaining dependency information for thermo from https://files.pythonhosted.org/packages/a4/b2/920e10c3f64882508c6200d6ead5452f68cbebc13ae16fc9b94512b0b09a/thermo-0.2.27-py3-none-any.whl.metadata
  Obtaining dependency information for chemicals>=1.1.5 from https://files.pythonhosted.org/packages/b2/df/a6bf8bb7d11a9dc1780bdeca30e1fedbdb4af3fd52d2510e33f86c0e00f1/chemicals-1.1.5-py3-none-any.whl.metadata
  Obtaining dependency information for fluids>=1.0.25 from https://files.pythonhosted.org/packages/d4/ca/803bddb96af07a8bac8d194b4699f6492dab141e1d593adf0ff611f142d5/fluids-1.0.25-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.4 MB 330.3 kB/s eta 0:00:32
   ---------------------------------------- 0.1/10.4 MB 563.7 kB/s eta 0:00:19
   - -------------------------------------- 0.4/10.4 MB 2.3 MB/s eta 0:00:05
   -- ----

In [7]:
# Set the conditions and imports
from scipy.constants import bar, hour
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CEOSGas, FlashPureVLS
fluid = 'R-22'
constants, correlations = ChemicalConstantsPackage.from_IDs([fluid])

T1 = 300.0
P1 = 1*bar
P2 = 5*bar
eta_isentropic = 0.7
eta_mechanical = 0.9

In [8]:
# Use the default originally published Peng-Robinson models
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas)
liquid = CEOSLiquid(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
gas = CEOSGas(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])

# Flash at inlet conditions to obtain initial enthalpy
state_1 = flasher.flash(T=T1, P=P1)
# Flash at outlet condition - entropy is conserved by compressors and expanders!
state_2_ideal = flasher.flash(S=state_1.S(), P=P2)
# Compute the change in enthalpy
delta_H_ideal = (state_2_ideal.H()-state_1.H())
# The definition of isentropic efficiency means that the actual amount of heat added is
# dH_actual = dH_idea/eta_isentropic
H_added_to_fluid_actual = delta_H_ideal/eta_isentropic

state_2 = flasher.flash(H=state_1.H() + H_added_to_fluid_actual, P=P2)

# To compute the actual power, itis more convinient to use the mass enthalpy
actual_power_per_kg = (state_2.H_mass() - state_1.H_mass())/(eta_mechanical) # W/kg
actual_power = actual_power_per_kg*100/hour
print(f'The actual power is {actual_power:.0f} W')
print(f'The actual outlet temperature is {state_2.T: .2f} K')

The actual power is 2251 W
The actual outlet temperature is  405.98 K
